In [1]:
import pandas as pd
from glob import glob as g
from pathlib import Path
import re
import numpy as np
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [2]:
def generate_uuid(row):
    try:
        uuid =  '_'.join(map(str, [row['date'].replace('-', '_'),
                row['timestamp'].replace(':', '_'),
                row['user'],
                str(row['file_ind'])]))
    except Exception as e:
        print(row['file_ind'])
        raise e
    return uuid

In [3]:
def fill_missing_annots(df):
    child_set = set(df['child'].tolist())
    temp_df = df[~df['parent'].isin(child_set)].copy()
    temp_df['child'] = temp_df['parent']
    temp_df = temp_df.drop_duplicates()
    return temp_df

In [4]:
class FileCleaner:
    """
    Contains various methods to clean a raw .txt file.
    """
    # Set patterns
    TIMESTAMP_PATTERN = "[[](\d\d:\d\d)[]]"
    USER_PATTERN = "[[]\d\d:\d\d[]]\s[<](.*?)[>]"
    MESSAGE_PATTERN = ".*[<].*[>](.*)"
    DATE_PATTERN = "(\d\d\d\d-\d\d-\d\d)"

    NOON = 720

    @staticmethod
    def load_file(file_path):
        """

        :param file_path: string of file path
        :return: pandas DataFrame object with line text as one column
        """
        return pd.read_csv(file_path, header = None, names = ['raw'], delimiter="\t")

    @staticmethod
    def generate_uuid(row):
        """
        Required column names in row: 'date', 'timestamp', 'user'
        :param row: pandas dataframe row
        :return: generated uuid from row parameters
        """
        try:
            uuid =  '_'.join(map(str, [row['date'].replace('-', '_'),
                    row['timestamp'].replace(':', '_'),
                    row['user'],
                    str(row['file_ind'])]))
        except Exception as e:
            print(row['file_ind'])
            raise e
        return uuid

    @staticmethod
    def extract_timestamp(raw_series):
        """

        :param raw_series: "raw" column of data
        :return: extracted timestamp Series
        """
        time_series = raw_series.str.extract(FileCleaner.TIMESTAMP_PATTERN, expand=True)
        #time_series = time_series.fillna("System")
        time_series = time_series.fillna(method='bfill')
        return time_series

    @staticmethod
    def convert_timestamp(raw_series):
        """
        calculate raw minutes of dataset
        :param raw_series: "raw" column of timestamp
        :return: series of integers
        """

        time_df = raw_series.str.split(':', expand=True)
        time_df = time_df.fillna(method='bfill')
        time_df = time_df.fillna(method='ffill')
        #time_df = time_df.fillna(-1)
        time_df['minutes_sum'] = time_df[0].astype(int) * 60 + time_df[1].astype(int)
        return time_df['minutes_sum']

    @staticmethod
    def extract_hour_minute(raw_series):
        time_df = raw_series.str.split(':', expand=True)
        time_df = time_df.fillna(method='bfill')
        time_df = time_df.fillna(method='ffill')
        time_df.columns = ['hour', 'minutes']
        return time_df

    @staticmethod
    def convert_12_to_24(raw_series):
        new_series = raw_series.copy()
        for ind, val in enumerate(raw_series.iloc[1:].values):
            if raw_series.iloc[ind - 1] > raw_series.iloc[ind]:
                new_series.update(raw_series.iloc[ind:] + FileCleaner.NOON)
                break
        return new_series

    @staticmethod
    def load_user_dict(user_dict_path, id_to_txt=False):
        """
        Loads user_dict csv and returns dictionary key:user_txt, value:user_id
        :param user_dict_path:
        :return: dictionary
        """
        map_df = pd.read_csv(user_dict_path, index_col=0)
        if id_to_txt:
            users_map = dict(zip(map_df['user_id'], map_df['user_txt']))
        else:
            users_map = dict(zip(map_df['user_txt'], map_df['user_id']))
        return users_map

    @staticmethod
    def extract_user(raw_series):
        """

        :param raw_series: Series of raw message text
        :return:
        """
        user_series = raw_series.str.extract(FileCleaner.USER_PATTERN, expand=True)
        user_series = user_series.fillna("System")
        return user_series

    @staticmethod
    def map_user(user_series, user_map_path):
        """
        Map string usernames to integer indices from users.csv
        :param user_series:
        :param user_map_path:
        :return:
        """
        map_df = pd.read_csv(user_map_path, index_col=0)
        users_map = dict(zip(map_df['user_txt'], map_df['user_id']))
        return user_series.map(users_map)

    @staticmethod
    def extract_mentions(raw_series, users_map):
        mentioned_users_series = raw_series.apply(lambda x: FileCleaner.retrieve_user_in_message(x, users_map))
        return mentioned_users_series

    @staticmethod
    def retrieve_user_in_message(message, users_map):
        """
        Helper method for extract_mentions to determine whether message contains a mention
        :param message:
        :return: user_ind
        """
        if not message:
            return -1
        DELIMITERS = [',', ':']
        for delim in DELIMITERS:
            potential_user = str(message).split(delim)[0]
            if potential_user in users_map:
                return users_map[potential_user]
        return -1
    @staticmethod
    def clean_file(file_path):
        #Load to dataframe
        data = pd.read_csv(file_path, header = None, names = ['raw'], delimiter="\t")

        data['file_ind'] = data.index.values
        data['file_ind'] = data['file_ind'].astype(int)

        # #Extract Timestamp
        # data['timestamp'] = data['raw'].str.extract(FileCleaner.TIMESTAMP_PATTERN, expand=True)
        # data.loc[data['timestamp'].isnull(), 'timestamp'] = "System"
        data['timestamp'] = FileCleaner.extract_timestamp(data['raw'])

        data['minutes'] = FileCleaner.convert_timestamp(data['timestamp'])
        data['minutes'] = FileCleaner.convert_12_to_24(data['minutes'])

        data['timestamp'] = data['timestamp'].fillna("System")
        data['user'] = FileCleaner.extract_user(data['raw'])

        map_path = '../data/cleaned/users.csv'
        data['user_ind'] = FileCleaner.map_user(data['user'], map_path)
        user_map = FileCleaner.load_user_dict(map_path, id_to_txt=False)

        data['message'] = data['raw'].str.extract(FileCleaner.MESSAGE_PATTERN, expand=True)
        data['message'] = data['message'].str.strip()

        data['mentions'] = FileCleaner.extract_mentions(data['message'], user_map)

        #data['date'] = Path(filename).stem.split('_')[0]
        temp_date = re.search(FileCleaner.DATE_PATTERN, file_path).group(1)
        data['date'] = temp_date

        data['uuid'] = data.apply(lambda row: FileCleaner.generate_uuid(row), axis=1)

        return data

In [4]:
# Set patterns
timestamp_pattern = "[[](\d\d:\d\d)[]]"
user_pattern = "[[]\d\d:\d\d[]]\s[<](.*?)[>]"
message_pattern = ".*[<].*[>](.*)"
date_pattern = "(\d\d\d\d-\d\d-\d\d)"
# Get files

# Train
filepath = '../data/'
subfolders = ["train", "test", "dev"]

file_list = []
raw_df_dict = {}
annot_dict = {}
for subfolder in subfolders:
    filelist_ascii = g(filepath + subfolder + '/*ascii.txt')
    #filelist_annot = g(filepath + subfolder + '/*annotation.txt')
    for filename in filelist_ascii:
        file_list.append(filename)
        filename_annot = filename.replace('ascii', 'annotation')
        data = pd.read_csv(filename, header = None, names = ['raw'], delimiter="\t")
        data['timestamp'] = data['raw'].str.extract(timestamp_pattern, expand=True)
        data.loc[data['timestamp'].isnull(), 'timestamp'] = "System"
        data['user'] = data['raw'].str.extract(user_pattern, expand=True)
        data.loc[data['user'].isnull(), 'user'] = "System"
        data['message'] = data['raw'].str.extract(message_pattern, expand=True)
        data['file_ind'] = data.index.values
        data['file_ind'] = data['file_ind'].astype(int)
        #data['date'] = Path(filename).stem.split('_')[0]
        temp_date = re.search(date_pattern, filename).group(1)
        data['date'] = temp_date
        data['uuid'] = data.apply(lambda row: generate_uuid(row), axis=1)

        raw_df_dict[temp_date] = data.copy()

        annot_df = pd.read_csv(filename_annot, index_col=False, header=None, names=['parent', 'child'], delimiter="\s")
        annot_df = pd.concat([annot_df, fill_missing_annots(annot_df)]).drop_duplicates()
        annot_dict[temp_date] = annot_df
        merged_data = pd.merge(data, annot_df, left_on='file_ind', right_on='child', how='left')

        #merged_data['parent_uuid'] = pd.merge(merged_data[['child', 'parent', 'uuid']], merged_data[['file_ind', 'child', 'parent', 'uuid']], left_on='parent', right_on='file_ind', how='left', suffixes=['_l', '_r'])['uuid_r']
        merged_data = pd.merge(merged_data, merged_data[['file_ind', 'uuid']], left_on='parent', right_on='file_ind', how='left', suffixes=['', '_parent'])
        #merged_data['parent_uuid'] = pd.merge(merged_data[['child', 'parent', 'uuid']], merged_data[['file_ind', 'child', 'parent', 'uuid']], left_on='parent', right_on='file_ind', how='left', suffixes=['_l', '_r'])['uuid_r']

        merged_data.to_csv(filename + "_annot.csv")

C:\Users\Daniel\anaconda3\envs\tf\lib\site-packages\pandas\core\strings\accessor.py:2403: FutureWarning: Possible nested set at position 1
  regex = re.compile(pat, flags=flags)
C:\Users\Daniel\anaconda3\envs\tf\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)
C:\Users\Daniel\anaconda3\envs\tf\lib\site-packages\pandas\io\parsers\readers.py:586: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return _read(filepath_or_buffer, kwds)


In [5]:
filepath = '../data/'
subfolders = ["dev", "train", "test"]

file_list = []
raw_df_dict = {}
annot_dict = {}
for subfolder in subfolders:
    filelist_ascii = g(filepath + subfolder + '/*ascii.txt')
    #filelist_annot = g(filepath + subfolder + '/*annotation.txt')
    for filename in filelist_ascii:
        file_list.append(filename)

In [6]:
#file_list

In [7]:
file_list[0]

'../data/dev\\2004-11-15_03.ascii.txt'

In [9]:
for file_path in file_list:
    filename_annot = file_path.replace('ascii', 'annotation')
    data = FileCleaner.clean_file(file_path)

    annot_df = pd.read_csv(filename_annot, index_col=False, header=None, names=['parent', 'child'], delimiter="\s")
    annot_df = pd.concat([annot_df, fill_missing_annots(annot_df)]).drop_duplicates()
    #annot_dict[temp_date] = annot_df
    merged_data = pd.merge(data, annot_df, left_on='file_ind', right_on='child', how='left')

    #merged_data['parent_uuid'] = pd.merge(merged_data[['child', 'parent', 'uuid']], merged_data[['file_ind', 'child', 'parent', 'uuid']], left_on='parent', right_on='file_ind', how='left', suffixes=['_l', '_r'])['uuid_r']
    merged_data = pd.merge(merged_data, merged_data[['file_ind', 'uuid']], left_on='parent', right_on='file_ind', how='left', suffixes=['', '_parent'])
    #merged_data['parent_uuid'] = pd.merge(merged_data[['child', 'parent', 'uuid']], merged_data[['file_ind', 'child', 'parent', 'uuid']], left_on='parent', right_on='file_ind', how='left', suffixes=['_l', '_r'])['uuid_r']

    merged_data.to_csv(file_path + "_annot.csv")

C:\Users\Daniel\anaconda3\envs\tf\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)
C:\Users\Daniel\anaconda3\envs\tf\lib\site-packages\pandas\io\parsers\readers.py:586: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return _read(filepath_or_buffer, kwds)
C:\Users\Daniel\anaconda3\envs\tf\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)
C:\Users\Daniel\anaconda3\envs\tf\lib\site

In [10]:
merged_data

,raw,file_ind,timestamp,minutes,user,user_ind,message,mentions,date,uuid,parent,child,file_ind_parent,uuid_parent
0,[21:16] <lestus> o/,0,21:16,1276,lestus,3017,o/,-1,2016-06-08,2016_06_08_21_16_lestus_0,NaN,NaN,NaN,NaN
1,"[21:16] <lordcirth> Guest21456, hi",1,21:16,1276,lordcirth,8265,"Guest21456, hi",6507,2016-06-08,2016_06_08_21_16_lordcirth_1,NaN,NaN,NaN,NaN
2,[21:16] <explosive> lorddoskias1: nope,2,21:16,1276,explosive,10529,lorddoskias1: nope,8911,2016-06-08,2016_06_08_21_16_explosive_2,NaN,NaN,NaN,NaN
3,"[21:16] <lorddoskias1> huhz, rather strange in...",3,21:16,1276,lorddoskias1,8911,"huhz, rather strange indeed",-1,2016-06-08,2016_06_08_21_16_lorddoskias1_3,NaN,NaN,NaN,NaN
4,[21:16] <Guest21456> I want to lean about Linux.,4,21:16,1276,Guest21456,6507,I want to lean about Linux.,-1,2016-06-08,2016_06_08_21_16_Guest21456_4,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1512,[13:34] <Xin> evening all,1495,13:34,1534,Xin,5754,evening all,-1,2016-06-08,2016_06_08_13_34_Xin_1495,1495.0,1495.0,1495.0,2016_06_08_13_34_Xin_1495
1513,[13:34] <ikonia> jimbotux: that sort of makes ...,1496,13:34,1534,ikonia,6068,jimbotux: that sort of makes sense,8102,2016-06-08,2016_06_08_13_34_ikonia_1496,1489.0,1496.0,1489.0,2016_06_08_13_33_jimbotux_1489
1514,[13:35] <sveinse> ikonia: But why does ubuntu ...,1497,13:35,1535,sveinse,593,ikonia: But why does ubuntu maintain /etc/rc*....,6068,2016-06-08,2016_06_08_13_35_sveinse_1497,1492.0,1497.0,1492.0,2016_06_08_13_34_ikonia_1492
1515,"[13:35] <ikonia> sveinse: yes, as some upstart...",1498,13:35,1535,ikonia,6068,"sveinse: yes, as some upstart scripts are wrapped",593,2016-06-08,2016_06_08_13_35_ikonia_1498,1497.0,1498.0,1497.0,2016_06_08_13_35_sveinse_1497


In [60]:
map_path = '../data/cleaned/users.csv'
user_map = FileCleaner.load_user_dict(map_path, id_to_txt=False)
FileCleaner.extract_mentions(merged_data['message'], user_map)

0      -1
1      -1
2      -1
3      -1
4      -1
       ..
1512   -1
1513   -1
1514   -1
1515   -1
1516   -1
Name: message, Length: 1517, dtype: int64

In [64]:
merged_data.iloc[2]['message']

' lorddoskias1: nope'

In [ ]:
user_map

In [82]:
annot_df

,parent,child
0,999,1000
1,1000,1001
2,995,1002
3,1002,1003
4,1002,1004
...,...,...
509,1497,1498
510,1496,1499
0,999,999
2,995,995


In [83]:
filename_annot

'../data/test\\2016-06-08_07.annotation.txt'

In [6]:
merged_data.columns.values

array(['raw', 'timestamp', 'user', 'message', 'file_ind', 'date', 'uuid',
       'parent', 'child', 'file_ind_parent', 'uuid_parent'], dtype=object)

In [11]:
# Train
filepath = '../data/train'
csvs_train = pd.DataFrame(g(filepath + '/*annot.csv'), columns=["path"])
#csvs_train["date"] = csvs_train["path"].str.extract(date_pattern, expand=True)

# Test
filepath = '../data/test'
csvs_test = pd.DataFrame(g(filepath + '/*annot.csv'), columns=["path"])
#csvs_test["date"] = csvs_test["path"].str.extract(date_pattern, expand=True)


# Dev
filepath = '../data/dev'
csvs_dev = pd.DataFrame(g(filepath + '/*annot.csv'), columns=["path"])
#csvs_dev["date"] = csvs_dev["path"].str.extract(date_pattern, expand=True)


In [14]:
#Aggregate files
rootfile = '../data/cleaned/'

agg_train = pd.DataFrame()
for i in range(len(csvs_train.path)):
    data = pd.read_csv(csvs_train.path[i], header = 0, index_col=0)
    # data['date'] = pd.Series([csvs_train.date[i] for x in range(len(data))])
    agg_train = agg_train.append(data)
agg_train.to_csv(rootfile + "agg_train.csv")

agg_test = pd.DataFrame()
for i in range(len(csvs_test.path)):
    data = pd.read_csv(csvs_test.path[i], header = 0, index_col=0)
    # data['date'] = pd.Series([csvs_test.date[i] for x in range(len(data))])
    agg_test = agg_test.append(data)
agg_test.to_csv(rootfile + "agg_test.csv")

agg_dev = pd.DataFrame()
for i in range(len(csvs_dev.path)):
    data = pd.read_csv(csvs_dev.path[i], header = 0, index_col=0)
    # data['date'] = pd.Series([csvs_dev.date[i] for x in range(len(data))])
    agg_dev = agg_dev.append(data)
agg_dev.to_csv(rootfile + "agg_dev.csv")

In [15]:
agg_dev

,raw,file_ind,timestamp,minutes,user,user_ind,message,mentions,date,uuid,parent,child,file_ind_parent,uuid_parent
0,"[12:18] <|trey|> usual, quite stable though :)",0,12:18,738,|trey|,10946,"usual, quite stable though :)",7183,2004-11-15,2004_11_15_12_18_|trey|_0,NaN,NaN,NaN,NaN
1,[12:18] <tweaked> HrdwrBoB: ok how many partit...,1,12:18,738,tweaked,1375,HrdwrBoB: ok how many partitions should i make?,6814,2004-11-15,2004_11_15_12_18_tweaked_1,NaN,NaN,NaN,NaN
2,"[12:18] <Matt|> |trey|, top in the list --> ub...",2,12:18,738,Matt|,6784,ubuntu servers,-1,2004-11-15,2004_11_15_12_18_Matt|_2,NaN,NaN,NaN,NaN
3,[12:18] <usual> a few libs and media,3,12:18,738,usual,7183,a few libs and media,-1,2004-11-15,2004_11_15_12_18_usual_3,NaN,NaN,NaN,NaN
4,[12:18] <usual> maybe some others,4,12:18,738,usual,7183,maybe some others,-1,2004-11-15,2004_11_15_12_18_usual_4,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,[21:57] <zacky83> who can help me on this,1245,21:57,2037,zacky83,14168,who can help me on this,-1,2016-12-19,2016_12_19_21_57_zacky83_1245,1244.0,1245.0,1244.0,2016_12_19_21_57_zacky83_1244
1260,"[21:57] <Mccallum1983> can anyone assist, when...",1246,21:57,2037,Mccallum1983,12323,"can anyone assist, when i try to install bitco...",-1,2016-12-19,2016_12_19_21_57_Mccallum1983_1246,1246.0,1246.0,1246.0,2016_12_19_21_57_Mccallum1983_1246
1261,[21:57] <figure002> OerHeks: still makes no se...,1247,21:57,2037,figure002,6694,OerHeks: still makes no sense to me why a daem...,15400,2016-12-19,2016_12_19_21_57_figure002_1247,1242.0,1247.0,1242.0,2016_12_19_21_56_OerHeks_1242
1262,[21:58] <figure002> zacky83: did you enable th...,1248,21:58,2038,figure002,6694,zacky83: did you enable the jails?,14168,2016-12-19,2016_12_19_21_58_figure002_1248,1244.0,1248.0,1244.0,2016_12_19_21_57_zacky83_1244


In [70]:
#agg_dev
# Remove command messages
agg_dict = {
    'dev': agg_dev,
    'train': agg_train,
    'test': agg_test
}

for key in agg_dict.keys():
    agg_dict[key] = agg_dict[key][agg_dict[key]['timestamp'].notnull()]

In [71]:
agg_dict['dev']

,raw,timestamp,minutes,user,user_ind,message,file_ind,date,uuid,parent,child,file_ind_parent,uuid_parent
0,"[12:18] <|trey|> usual, quite stable though :)",12:18,738,|trey|,10946,"usual, quite stable though :)",0,2004-11-15,2004_11_15_12_18_|trey|_0,NaN,NaN,NaN,NaN
1,[12:18] <tweaked> HrdwrBoB: ok how many partit...,12:18,738,tweaked,1375,HrdwrBoB: ok how many partitions should i make?,1,2004-11-15,2004_11_15_12_18_tweaked_1,NaN,NaN,NaN,NaN
2,"[12:18] <Matt|> |trey|, top in the list --> ub...",12:18,738,Matt|,6784,ubuntu servers,2,2004-11-15,2004_11_15_12_18_Matt|_2,NaN,NaN,NaN,NaN
3,[12:18] <usual> a few libs and media,12:18,738,usual,7183,a few libs and media,3,2004-11-15,2004_11_15_12_18_usual_3,NaN,NaN,NaN,NaN
4,[12:18] <usual> maybe some others,12:18,738,usual,7183,maybe some others,4,2004-11-15,2004_11_15_12_18_usual_4,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1259,[21:57] <zacky83> who can help me on this,21:57,1317,zacky83,14168,who can help me on this,1245,2016-12-19,2016_12_19_21_57_zacky83_1245,1244.0,1245.0,1244.0,2016_12_19_21_57_zacky83_1244
1260,"[21:57] <Mccallum1983> can anyone assist, when...",21:57,1317,Mccallum1983,12323,"can anyone assist, when i try to install bitc...",1246,2016-12-19,2016_12_19_21_57_Mccallum1983_1246,1246.0,1246.0,1246.0,2016_12_19_21_57_Mccallum1983_1246
1261,[21:57] <figure002> OerHeks: still makes no se...,21:57,1317,figure002,6694,OerHeks: still makes no sense to me why a dae...,1247,2016-12-19,2016_12_19_21_57_figure002_1247,1242.0,1247.0,1242.0,2016_12_19_21_56_OerHeks_1242
1262,[21:58] <figure002> zacky83: did you enable th...,21:58,1318,figure002,6694,zacky83: did you enable the jails?,1248,2016-12-19,2016_12_19_21_58_figure002_1248,1244.0,1248.0,1244.0,2016_12_19_21_57_zacky83_1244


In [11]:
# for key in agg_dict.keys():
#     agg_dict[key]['uuid'] = agg_dict[key].apply(lambda row: generate_uuid(row), axis=1)


In [12]:
agg_dict['dev'][agg_dict['dev']['uuid'] == "2004_11_15_01_35_djtansey_685"]

,raw,timestamp,user,message,file_ind,date,uuid,parent,child,file_ind_parent,uuid_parent
685,[01:35] <djtansey> i have a problem re: k3b an...,01:35,djtansey,i have a problem re: k3b and am looking for s...,685,2004-11-15,2004_11_15_01_35_djtansey_685,685.0,685.0,685.0,2004_11_15_01_35_djtansey_685


In [13]:
for key in agg_dict.keys():
    agg_dict[key].to_csv(rootfile + f"data/cleaned/agg_{key}.csv")

In [14]:
data = raw_df_dict['2005-07-06']
annot_df = annot_dict['2005-07-06']
annot_df = pd.concat([annot_df, fill_missing_annots(annot_df)], ignore_index=True).drop_duplicates()
merged_data = pd.merge(data, annot_df, left_on='file_ind', right_on='child', how='left')

#merged_data['parent_uuid'] = pd.merge(merged_data[['child', 'parent', 'uuid']], merged_data[['file_ind', 'child', 'parent', 'uuid']], left_on='parent', right_on='file_ind', how='left', suffixes=['_l', '_r'])['uuid_r']
merged_data = pd.merge(merged_data, merged_data[['file_ind', 'uuid']], left_on='parent', right_on='file_ind', how='left', suffixes=['', '_parent'])
#merged_data.to_csv(filename + "_annot.csv")

In [15]:
temp = pd.merge(merged_data[['parent']], merged_data[['file_ind', 'uuid']], left_on='parent', right_on='file_ind', how='left', suffixes=['_l', '_r'])
temp

,parent,file_ind,uuid
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,NaN,NaN,NaN
...,...,...,...
1516,1493.0,1493.0,2005_07_06_03_35_Nige_1493
1517,1495.0,1495.0,2005_07_06_03_35_Nige_1495
1518,1495.0,1495.0,2005_07_06_03_35_Nige_1495
1519,1498.0,1498.0,2005_07_06_03_35_FLD_1498


In [16]:
merged_data = pd.merge(merged_data, merged_data[['file_ind', 'uuid']], left_on='parent', right_on='file_ind', how='left', suffixes=['', '_parent'])

In [17]:
merged_data

,raw,timestamp,user,message,file_ind,date,uuid,parent,child,file_ind_parent,uuid_parent,file_ind_parent,uuid_parent
0,"[11:11] <Seveas> Amaranth, the US peer is prob...",11:11,Seveas,"Amaranth, the US peer is probably breaking th...",0,2005-07-06,2005_07_06_11_11_Seveas_0,NaN,NaN,NaN,NaN,NaN,NaN
1,[11:11] <Seveas> but an ES or NL peer download...,11:11,Seveas,but an ES or NL peer downloading it not,1,2005-07-06,2005_07_06_11_11_Seveas_1,NaN,NaN,NaN,NaN,NaN,NaN
2,[11:11] <monchichi> http://www.msnbc.msn.com/i...,11:11,monchichi,http://www.msnbc.msn.com/id/8419601/,2,2005-07-06,2005_07_06_11_11_monchichi_2,NaN,NaN,NaN,NaN,NaN,NaN
3,[11:11] <IceDC571> the US peer is just stupid ...,11:11,IceDC571,the US peer is just stupid for wanting to sha...,3,2005-07-06,2005_07_06_11_11_IceDC571_3,NaN,NaN,NaN,NaN,NaN,NaN
4,[11:11] <Seveas> The downloader is breaking th...,11:11,Seveas,The downloader is breaking the implicit rules...,4,2005-07-06,2005_07_06_11_11_Seveas_4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1516,[03:35] <Nige> i am stuck with wireless networ...,03:35,Nige,i am stuck with wireless networking,1495,2005-07-06,2005_07_06_03_35_Nige_1495,1493.0,1495.0,1493.0,2005_07_06_03_35_Nige_1493,1493.0,2005_07_06_03_35_Nige_1493
1517,[03:35] <Nige> :(,03:35,Nige,:(,1496,2005-07-06,2005_07_06_03_35_Nige_1496,1495.0,1496.0,1495.0,2005_07_06_03_35_Nige_1495,1495.0,2005_07_06_03_35_Nige_1495
1518,[03:35] <Nige> and its driving me crazy!!!,03:35,Nige,and its driving me crazy!!!,1497,2005-07-06,2005_07_06_03_35_Nige_1497,1495.0,1497.0,1495.0,2005_07_06_03_35_Nige_1495,1495.0,2005_07_06_03_35_Nige_1495
1519,[03:35] <FLD> does anybody know how to get dsn...,03:35,FLD,does anybody know how to get dsniff to work :<,1498,2005-07-06,2005_07_06_03_35_FLD_1498,1498.0,1498.0,1498.0,2005_07_06_03_35_FLD_1498,1498.0,2005_07_06_03_35_FLD_1498


In [18]:
annot_df

,parent,child
0,993,1000
1,995,1000
2,1001,1001
3,1000,1002
4,998,1003
...,...,...
504,1498,1498
505,1494,1499
506,993,993
507,995,995


In [163]:
annot_df[annot_df['child']==1262]

,parent,child
265,1254,1262


In [164]:
data[data['file_ind'] == 1262]

,raw,timestamp,user,message,file_ind,date,uuid
1262,"[02:59] <Vjaz> delire, Yeah, I'm no stranger t...",02:59,Vjaz,"delire, Yeah, I'm no stranger to compiling my...",1262,2005-07-06,2005_07_06_02_59_Vjaz_1262


In [188]:
merged_data = pd.merge(data, annot_df, left_on='file_ind', right_on='child', how='left')
merged_data

,raw,timestamp,user,message,file_ind,date,uuid,parent,child
0,"[11:11] <Seveas> Amaranth, the US peer is prob...",11:11,Seveas,"Amaranth, the US peer is probably breaking th...",0,2005-07-06,2005_07_06_11_11_Seveas_0,NaN,NaN
1,[11:11] <Seveas> but an ES or NL peer download...,11:11,Seveas,but an ES or NL peer downloading it not,1,2005-07-06,2005_07_06_11_11_Seveas_1,NaN,NaN
2,[11:11] <monchichi> http://www.msnbc.msn.com/i...,11:11,monchichi,http://www.msnbc.msn.com/id/8419601/,2,2005-07-06,2005_07_06_11_11_monchichi_2,NaN,NaN
3,[11:11] <IceDC571> the US peer is just stupid ...,11:11,IceDC571,the US peer is just stupid for wanting to sha...,3,2005-07-06,2005_07_06_11_11_IceDC571_3,NaN,NaN
4,[11:11] <Seveas> The downloader is breaking th...,11:11,Seveas,The downloader is breaking the implicit rules...,4,2005-07-06,2005_07_06_11_11_Seveas_4,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1201,[03:35] <Nige> i am stuck with wireless networ...,03:35,Nige,i am stuck with wireless networking,1495,2005-07-06,2005_07_06_03_35_Nige_1495,1493.0,1495.0
1202,[03:35] <Nige> :(,03:35,Nige,:(,1496,2005-07-06,2005_07_06_03_35_Nige_1496,1495.0,1496.0
1203,[03:35] <Nige> and its driving me crazy!!!,03:35,Nige,and its driving me crazy!!!,1497,2005-07-06,2005_07_06_03_35_Nige_1497,1495.0,1497.0
1204,[03:35] <FLD> does anybody know how to get dsn...,03:35,FLD,does anybody know how to get dsniff to work :<,1498,2005-07-06,2005_07_06_03_35_FLD_1498,1498.0,1498.0


In [168]:
merged_data['parent_uuid'] = pd.merge(merged_data[['child', 'parent', 'uuid']], merged_data[['file_ind', 'child', 'parent', 'uuid']], left_on='parent', right_on='file_ind', how='left', suffixes=['_l', '_r'])['uuid_r']

In [185]:
merged_data

,raw,timestamp,user,message,file_ind,date,uuid,parent_x,child_x,parent_uuid,parent_y,child_y
0,[04:14] <Gobbert> ziggi: what do you need help...,04:14,Gobbert,ziggi: what do you need help with?,0,2016-12-19,2016_12_19_04_14_Gobbert_0,NaN,NaN,NaN,NaN,NaN
1,[04:14] <ziggi> i am,04:14,ziggi,i am,1,2016-12-19,2016_12_19_04_14_ziggi_1,NaN,NaN,NaN,NaN,NaN
2,[04:15] <joshua__> boot speed was very slow,04:15,joshua__,boot speed was very slow,2,2016-12-19,2016_12_19_04_15_joshua___2,NaN,NaN,NaN,NaN,NaN
3,[04:15] <joshua__> 2 min to boot,04:15,joshua__,2 min to boot,3,2016-12-19,2016_12_19_04_15_joshua___3,NaN,NaN,NaN,NaN,NaN
4,[04:15] <joshua__> but windows machine was ver...,04:15,joshua__,but windows machine was very fast,4,2016-12-19,2016_12_19_04_15_joshua___4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1205,[21:57] <zacky83> who can help me on this,21:57,zacky83,who can help me on this,1245,2016-12-19,2016_12_19_21_57_zacky83_1245,1244.0,1245.0,2016_12_19_21_55_zh19970205_1239,1244.0,1245.0
1206,"[21:57] <Mccallum1983> can anyone assist, when...",21:57,Mccallum1983,"can anyone assist, when i try to install bitc...",1246,2016-12-19,2016_12_19_21_57_Mccallum1983_1246,1246.0,1246.0,2016_12_19_21_55_zh19970205_1239,1246.0,1246.0
1207,[21:57] <figure002> OerHeks: still makes no se...,21:57,figure002,OerHeks: still makes no sense to me why a dae...,1247,2016-12-19,2016_12_19_21_57_figure002_1247,1242.0,1247.0,2016_12_19_21_55_zh19970205_1240,1242.0,1247.0
1208,[21:58] <figure002> zacky83: did you enable th...,21:58,figure002,zacky83: did you enable the jails?,1248,2016-12-19,2016_12_19_21_58_figure002_1248,1244.0,1248.0,2016_12_19_21_52_figure002_1231,1244.0,1248.0


In [189]:
merged_data[merged_data['child'] == 1262]

,raw,timestamp,user,message,file_ind,date,uuid,parent,child
1019,"[02:59] <Vjaz> delire, Yeah, I'm no stranger t...",02:59,Vjaz,"delire, Yeah, I'm no stranger to compiling my...",1262,2005-07-06,2005_07_06_02_59_Vjaz_1262,1254.0,1262.0


In [190]:
merged_data[merged_data['parent'] == 1262]

,raw,timestamp,user,message,file_ind,date,uuid,parent,child
1022,[03:00] <delire> Vjaz: it is a little. i think...,03:00,delire,Vjaz: it is a little. i think the best approa...,1265,2005-07-06,2005_07_06_03_00_delire_1265,1262.0,1265.0


In [197]:
merged_data

,raw,timestamp,user,message,file_ind,date,uuid,parent,child
0,"[11:11] <Seveas> Amaranth, the US peer is prob...",11:11,Seveas,"Amaranth, the US peer is probably breaking th...",0,2005-07-06,2005_07_06_11_11_Seveas_0,NaN,NaN
1,[11:11] <Seveas> but an ES or NL peer download...,11:11,Seveas,but an ES or NL peer downloading it not,1,2005-07-06,2005_07_06_11_11_Seveas_1,NaN,NaN
2,[11:11] <monchichi> http://www.msnbc.msn.com/i...,11:11,monchichi,http://www.msnbc.msn.com/id/8419601/,2,2005-07-06,2005_07_06_11_11_monchichi_2,NaN,NaN
3,[11:11] <IceDC571> the US peer is just stupid ...,11:11,IceDC571,the US peer is just stupid for wanting to sha...,3,2005-07-06,2005_07_06_11_11_IceDC571_3,NaN,NaN
4,[11:11] <Seveas> The downloader is breaking th...,11:11,Seveas,The downloader is breaking the implicit rules...,4,2005-07-06,2005_07_06_11_11_Seveas_4,NaN,NaN
...,...,...,...,...,...,...,...,...,...
1201,[03:35] <Nige> i am stuck with wireless networ...,03:35,Nige,i am stuck with wireless networking,1495,2005-07-06,2005_07_06_03_35_Nige_1495,1493.0,1495.0
1202,[03:35] <Nige> :(,03:35,Nige,:(,1496,2005-07-06,2005_07_06_03_35_Nige_1496,1495.0,1496.0
1203,[03:35] <Nige> and its driving me crazy!!!,03:35,Nige,and its driving me crazy!!!,1497,2005-07-06,2005_07_06_03_35_Nige_1497,1495.0,1497.0
1204,[03:35] <FLD> does anybody know how to get dsn...,03:35,FLD,does anybody know how to get dsniff to work :<,1498,2005-07-06,2005_07_06_03_35_FLD_1498,1498.0,1498.0


In [22]:
time_df = merged_data['timestamp'].str.split(':', expand=True)
time_df = time_df.fillna(method='bfill')
time_df = time_df.fillna(-1)
time_df['minutes_sum'] = time_df[0].astype(int) * 60 + time_df[1].astype(int)

ValueError: invalid literal for int() with base 10: 'System'

In [192]:
temp = pd.merge(merged_data[['child', 'parent', 'uuid']], merged_data[['file_ind', 'child', 'parent', 'uuid']], left_on='parent', right_on='file_ind', how='left', suffixes=['_l', '_r']).copy()

In [194]:
temp[temp['child_l'] == 1262]

,child_l,parent_l,uuid_l,file_ind,child_r,parent_r,uuid_r
1020,1262.0,1254.0,2005_07_06_02_59_Vjaz_1262,1254.0,1254.0,1252.0,2005_07_06_02_58_delire_1254


In [198]:
merged_data['parent_uuid'] = pd.merge(merged_data[['child', 'parent', 'uuid']], merged_data[['file_ind', 'child', 'parent', 'uuid']], left_on='parent', right_on='file_ind', how='left', suffixes=['_l', '_r'])['uuid_r']

In [201]:
merged_data[['file_ind', 'date', 'uuid', 'parent', 'parent_uuid']]

,file_ind,date,uuid,parent,parent_uuid
0,0,2005-07-06,2005_07_06_11_11_Seveas_0,NaN,NaN
1,1,2005-07-06,2005_07_06_11_11_Seveas_1,NaN,NaN
2,2,2005-07-06,2005_07_06_11_11_monchichi_2,NaN,NaN
3,3,2005-07-06,2005_07_06_11_11_IceDC571_3,NaN,NaN
4,4,2005-07-06,2005_07_06_11_11_Seveas_4,NaN,NaN
...,...,...,...,...,...
1201,1495,2005-07-06,2005_07_06_03_35_Nige_1495,1493.0,2005_07_06_03_35_mindmedic_1490
1202,1496,2005-07-06,2005_07_06_03_35_Nige_1496,1495.0,2005_07_06_03_35_fdr_1492
1203,1497,2005-07-06,2005_07_06_03_35_Nige_1497,1495.0,2005_07_06_03_35_delire_1489
1204,1498,2005-07-06,2005_07_06_03_35_FLD_1498,1498.0,2005_07_06_03_34_wizo_1487


In [202]:
temp

,child_l,parent_l,uuid_l,file_ind,child_r,parent_r,uuid_r
0,NaN,NaN,2005_07_06_11_11_Seveas_0,NaN,NaN,NaN,NaN
1,NaN,NaN,2005_07_06_11_11_Seveas_1,NaN,NaN,NaN,NaN
2,NaN,NaN,2005_07_06_11_11_monchichi_2,NaN,NaN,NaN,NaN
3,NaN,NaN,2005_07_06_11_11_IceDC571_3,NaN,NaN,NaN,NaN
4,NaN,NaN,2005_07_06_11_11_Seveas_4,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1205,1495.0,1493.0,2005_07_06_03_35_Nige_1495,1493.0,1493.0,1489.0,2005_07_06_03_35_Nige_1493
1206,1496.0,1495.0,2005_07_06_03_35_Nige_1496,1495.0,1495.0,1493.0,2005_07_06_03_35_Nige_1495
1207,1497.0,1495.0,2005_07_06_03_35_Nige_1497,1495.0,1495.0,1493.0,2005_07_06_03_35_Nige_1495
1208,1498.0,1498.0,2005_07_06_03_35_FLD_1498,1498.0,1498.0,1498.0,2005_07_06_03_35_FLD_1498


In [139]:
raw_df_dict.keys()

dict_keys(['2004-12-25', '2005-02-06', '2005-02-08', '2005-02-27', '2005-05-14', '2005-05-19', '2005-06-06', '2005-06-12', '2005-06-16', '2005-06-20', '2005-07-25', '2005-07-29', '2005-09-26', '2005-10-07', '2005-10-12', '2005-12-03', '2005-12-04', '2005-12-16', '2005-12-23', '2006-01-02', '2006-01-12', '2006-02-20', '2006-02-24', '2006-02-28', '2006-03-05', '2006-05-02', '2006-05-15', '2006-05-27', '2006-05-29', '2006-06-01', '2006-06-05', '2006-06-08', '2006-06-21', '2006-06-28', '2006-07-01', '2006-08-06', '2006-08-11', '2006-08-13', '2006-08-15', '2006-08-23', '2006-09-13', '2006-09-24', '2006-11-01', '2006-12-06', '2006-12-10', '2006-12-20', '2007-01-12', '2007-01-19', '2007-01-21', '2007-01-29', '2007-02-06', '2007-02-07', '2007-02-15', '2007-06-01', '2007-06-04', '2007-06-17', '2007-07-03', '2007-08-19', '2007-08-22', '2007-08-24', '2007-09-07', '2007-10-24', '2007-12-17', '2008-01-02', '2008-01-03', '2008-02-07', '2008-02-14', '2008-03-01', '2008-04-20', '2008-04-27', '2008-04-

In [80]:
annot_df

,parent,child
0,999,1000
1,1001,1002
2,1001,1004
3,1007,1008
4,1007,1009
...,...,...
255,1044,1047
256,1084,1084
257,1172,1172
258,999,999


In [25]:
user_map_path = '../data/cleaned/users.csv'
map_df = pd.read_csv(user_map_path, index_col=0)
users_map = dict(zip(map_df['user_txt'], map_df['user_id']))

In [28]:
merged_data['user'].map(users_map)

0       15358
1       15358
2        7927
3       16253
4       15358
        ...  
1516     3459
1517     3459
1518     3459
1519    10757
1520    10875
Name: user, Length: 1521, dtype: int64